In [ ]:
import cadquery as cq
import math

In [ ]:
n = 3
w = 2
s = 1
d_out = 50
d_tmp = d_out
metal_height = 2
height = 10

x = 0
y = 0
outline = []

In [ ]:

outline.append((x,y))
x,y = d_out+x,y
outline.append((x,y))
x,y = outline[len(outline)-1][0],d_out + y
outline.append((x,y))
x,y = (outline[len(outline)-1][0])-d_out,outline[len(outline)-1][1]
outline.append((x,y))


orientation = ["down","right","up","left"]
count = 2

for _ in range(n-1):
    for direction in orientation:
        if count % 2 == 0:
            d_out = d_out - (w+s)
            count = 0
        if direction == "down":
            x, y = x, y - d_out
        elif direction == "up":
            x, y = x, y + d_out
        elif direction == "right":
            x, y = x + d_out, y
        elif direction == "left":
            x, y = x - d_out, y
        count += 1
        outline.append((x,y))



d_out = d_out - (w+s)


x,y = x,y - ((d_tmp / 2 - ((n-1)*w+(n-1)*s)) + (w / 2))
x_down,y_down = x,y
outline.append((x,y))
x, y = x + w, y
outline.append((x,y))
d_out = d_out - w
x,y = x,y + ( (d_tmp / 2 - ((n-1)*w+(n-1)*s)) + (w / 2)) - w
outline.append((x,y))



orientation = ["right","down","left","up"]

count = 0
d_out = d_out +s

for _ in range(n-1):
    for direction in orientation:
        if direction == "down":
            x, y = x, y - d_out
        elif direction == "up":
            x, y = x, y + d_out
        elif direction == "right":
            x, y = x + d_out, y
        elif direction == "left":
            x, y = x - d_out, y
        count += 1
        if count % 2 == 0:
            d_out = d_out + (w+s)
            count = 0
        outline.append((x,y))


x, y = x + d_out, y
outline.append((x,y))
x, y = x, y - d_out
outline.append((x,y))
d_out = d_out + w
x, y = x - d_out, y
outline.append((x,y))
x, y = x, y - w
outline.append((x,y))


print(outline)

In [ ]:
# To rotate an object
def rotate(arr, n, x_pivot, y_pivot, angle):
    sin = int(math.sin(angle * math.pi / 180))
    cos = int(math.cos(angle * math.pi / 180))
    shifted_points = [(x - x_pivot, y - y_pivot, z) if z else (x - x_pivot, y - y_pivot) for x, y, *z in arr]

    rotated_points = [(x_pivot + (x * cos - y * sin), y_pivot + (x * sin + y * cos), z) \
                    if z \
                    else (x_pivot + (x * cos - y * sin), y_pivot + (x * sin + y * cos)) for x, y, *z in shifted_points]
    return rotated_points


print(rotate(outline,n,0,0,90))

In [ ]:
# frame 2
outline2 = [(x,((-1)*y) + d_tmp,-height) for x,y in outline]
#print(outline2)

# frame 3
outline3 = [(x,y + d_tmp,-height*2) for x,y in rotate(outline,n,0,0,-90)]
#print(outline3)
end_outline3 = outline3[len(outline3) // 2]
#print(outline3[(len(outline3) // 2)])

# frame 4
outline4 = [(((-1)*x + d_tmp),y + d_tmp,-height*(3)) for x,y in rotate(outline,n,0,0,-90)]
#outline4 = [(((-1)*x),y,-height) for x,y in rotate(outline,n,0,0,-90)]
start_outline4 = outline4[0]
#print(outline4)

# frame 5
outline5 = [((x + d_tmp),y + d_tmp,-height*(4)) for x,y in rotate(outline,n,0,0,-180)]

# frame 6


#frame 7


# frame 8



In [ ]:
result = (
    cq.Workplane()
    .polyline(outline) # first frame
    .close()
    .extrude(metal_height)
    .faces('<Z') # get end of outline face
    .moveTo(x_down + w / 2,y_down + w / 2) # move to vias start point
    .rect(w,w) # draw vias
    .extrude(-height) # extrude vias
    .faces('<Z') # get vias's Z face
    .polyline(outline2) # second frame starting from vias face
    .close()
    .extrude(2)
    .faces("<Z") # get face of frame
    .workplane()
    .moveTo(outline2[-1][0] + w / 2,-outline2[-1][1] + w / 2) # vias
    .rect(w,w)
    .extrude(height)
    .faces('>Z') # get vias face
    .workplane()
    .polyline(outline3) # frame 3
    .close()
    .extrude(-metal_height)
    .faces("<Z")
    .workplane()
    .moveTo(end_outline3[0] + w / 2,-end_outline3[1] - w / 2) # vias
    .rect(w,w)
    .extrude(height)
    .faces('>Z')
    .workplane()
    .polyline(outline4)
    .close()
    .extrude(-metal_height)
    .faces('<Z') # vias 4
    .workplane()
    .moveTo(outline4[-1][0] - w / 2,-outline4[-1][1] + w / 2)
    .rect(w,w)
    .extrude(height)
    .faces('>Z')
    .workplane()
    .polyline(outline5)
    .close()
    .extrude(-metal_height)

    )
result

In [ ]:
result5 = cq.Workplane().polyline(rotate(outline,n,0,0,-180)).close().extrude(-5) #5. slika
result5

In [ ]:
cq.exporters.export(result, 'object.step')

In [ ]:
cq.importers.importStep("object.step")